In [1]:
import fire
import os
import sys
import time
import json
import re
import random
import logging

import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from vllm import LLM, SamplingParams

from inference_utils import get_user_prompt, post_processing_answer

In [2]:
def main(
    model_path,
    # model_name,
    # peft_model: str = None,
    quantization: bool = False,
    load_in: str = "4bit",
    max_new_tokens=100,  # The maximum numbers of tokens to generate
    test_file: str = "datasets/math_test.json",
    seed: int = 42,  # seed value for reproducibility
    do_sample: bool = True,  # Whether or not to use sampling ; use greedy decoding otherwise.
    min_length: int = None,  # The minimum length of the sequence to be generated, input prompt + min_new_tokens
    use_cache: bool = True,  # [optional] Whether or not the model should use the past last key/values attentions Whether or not the model should use the past last key/values attentions (if applicable to the model) to speed up decoding.
    top_p: float = 1.0,  # [optional] If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
    temperature: float = 1.0,  # [optional] The value used to modulate the next token probabilities.
    top_k: int = 50,  # [optional] The number of highest probability vocabulary tokens to keep for top-k-filtering.
    repetition_penalty: float = 1.0,  # The parameter for repetition penalty. 1.0 means no penalty.
    length_penalty: int = 1,  # [optional] Exponential penalty to the length that is used with beam-based generation.
    enable_azure_content_safety: bool = False,  # Enable safety check with Azure content safety api
    enable_sensitive_topics: bool = False,  # Enable check for sensitive topics using AuditNLG APIs
    enable_salesforce_content_safety: bool = True,  # Enable safety check with Salesforce safety flan t5
    max_padding_length: int = None,  # the max padding length to be used with tokenizer padding the prompts.
    use_fast_kernels: bool = False,  # Enable using SDPA from PyTroch Accelerated Transformers, make use Flash Attention and Xformer memory-efficient kernels
    log_filename: str = "log.txt",
    output_filepath: str = "jupyter_submission.csv",
    time_output_filepath: str = "time_submission.csv",
    **kwargs,
):
    logging.basicConfig(
        filename=log_filename,
        filemode='a',
        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
        datefmt='%H:%M:%S',
        level=logging.DEBUG
    )

    log = logging.getLogger(__name__)
    with open(test_file) as f:
        data = json.load(f)["data"]

    # Set the seeds for reproducibility
    torch.cuda.manual_seed(seed)
    torch.manual_seed(seed)

    model = LLM(model=model_path, seed=seed)
    sampling_params = SamplingParams(n=1, best_of=1, temperature=temperature, top_p=top_p, stop_token_ids=[2], max_tokens=max_new_tokens)

    if use_fast_kernels:
        """
        Setting 'use_fast_kernels' will enable
        using of Flash Attention or Xformer memory-efficient kernels
        based on the hardware being used. This would speed up inference when used for batched inputs.
        """
        try:
            from optimum.bettertransformer import BetterTransformer

            model = BetterTransformer.transform(model)
        except ImportError:
            log.error(
                "Module 'optimum' not found. Please install 'optimum' it before proceeding."
            )
    # Dummy
    model.generate(get_user_prompt(data[0]), sampling_params)
    
    results = []
    times = []

    for idx, example in enumerate(data):
        log.info(f"Processing {idx}")
        start = time.time()
        user_prompt = get_user_prompt(example)
        id = example["id"]
        choices = example["choices"]

        output = model.generate(user_prompt, sampling_params)[0]

        prompt = output.prompt
        gen_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {gen_text!r}")

        answer_text = None

        for text in gen_text.split("###"):
            if 'Final choice' in text:
                answer_text = text
                break

        if answer_text is None:
            answer_text = gen_text

        log.info(f"Output text: {prompt + gen_text}")
        log.info(f"Gen text {gen_text}")
        log.info(f"Answer text {answer_text}")

        answer = post_processing_answer(answer_text, choices)
        e2e_inference_time = int((time.time() - start) * 1000)
        log.info(f'Inference time: {e2e_inference_time}')
        log.info(f"Answer {answer}")
        if answer is None:
            answer = random.choice(choices)
            log.info(f"Random Answer {answer}")
        results.append({"id": id, "answer": answer})
        times.append({"id": id, "time": e2e_inference_time})

    result_df = pd.DataFrame.from_dict(results)
    result_df.to_csv(output_filepath, index=False)

    time_df = pd.DataFrame.from_dict(times)
    time_df.to_csv(time_output_filepath, index=False)

In [ ]:
main(
    model_path='final',
    max_new_tokens=1024,
    temperature=0.1
)

INFO 11-30 14:56:23 llm_engine.py:72] Initializing an LLM engine with config: model='final', tokenizer='final', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=42)
INFO 11-30 14:56:35 llm_engine.py:207] # GPU blocks: 2016, # CPU blocks: 2048


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.47s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\n### Question: Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?\n### Choices: ['A. 4 500 000 đồng','B. 45 000 000 đồng','C. 50 000 000 đồng','D. 450 000 000 đồng']\n### Explanation: ", Generated text: 'Đầu tiên, chúng ta cần hiểu rằng 30% của một số là một phần của số đó. Để tìm số đó, chúng ta cần chia 15 000 000 đồng cho 30%.\n\nĐể thực hiện phép chia này, chúng ta cần chuyển 15 000 000 đồng thành phần trăm. Để làm điều này, chúng ta chia 15 000 000 đồng cho 100 và nhân kết quả với 100% để biểu thị kết quả dưới dạng phần trăm.\n\nKhi thực hiện phép chia này, chúng ta được kết quả là 50 000 000 đồng.\n\nVì vậy, nếu bán hết hàng thì cửa hàng thu được 50 000 000 đồng.\n### Final choice: C. 50 000 000 đồng\n'
full answer process C. 50 000 000 đồng


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.18s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\n### Question: Một người đi xe đạp từ A lúc 7 giờ với vận tốc 12km/h. Đến 8 giờ một người đi xe máy cũng từ A đuổi theo người đi xe đạp với vận tốc 42km/h. Hỏi người đi xe máy đuổi kịp người đi xe đạp lúc mấy giờ?\n### Choices: ['A. 24 phút','B. 1 giờ','C. 7 giờ 24 phút','D. 8 giờ 24 phút']\n### Explanation: ", Generated text: 'Để giải quyết câu hỏi này, chúng ta cần hiểu rằng người đi xe đạp sẽ đi được 12km trong một giờ và người đi xe máy sẽ đi được 42km trong một giờ. Điều này có nghĩa là sau mỗi giờ, người đi xe máy sẽ đi được 30km hơn người đi xe đạp.\n\nĐể tìm thời gian mà người đi xe máy đuổi kịp người đi xe đạp, chúng ta cần chia số km mà người đi xe máy đi được trong một giờ cho sự khác biệt về vận tốc giữa hai người. \n\nNgười đi xe máy đi được 42km trong một giờ và người đi xe đạp đi được 12km trong một giờ. Điều này

...

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.73s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\n### Question: Lớp 5/2 có 32 học sinh, trong đó có 12 học sinh xuất sắc. Tỉ số phần trăm của số học sinh xuất sắc và học sinh cả lớp là:\n### Choices: ['A. 375 %','B. 37,5 %','C. 3,75 %','D. 0,375 %']\n### Explanation: ", Generated text: 'Để giải quyết câu hỏi này, chúng ta cần hiểu rằng tỉ số phần trăm là một cách để biểu thị một quan hệ giữa hai số. Trong trường hợp này, chúng ta cần tìm tỉ số phần trăm của số học sinh xuất sắc và số học sinh cả lớp.\n\nBắt đầu bằng cách xác định số học sinh cả lớp. Trong trường hợp này, chúng ta biết rằng lớp 5/2 có 32 học sinh.\n\nTiếp theo, chúng ta cần xác định số học sinh xuất sắc. Trong trường hợp này, chúng ta biết rằng có 12 học sinh xuất sắc.\n\nCuối cùng, chúng ta cần chia số học sinh xuất sắc cho số học sinh cả lớp và nhân với 100% để biểu thị kết quả dưới dạng phần trăm.\n\nVậy, t

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.11s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\n### Question: Số thập phân thích hợp để điền vào chỗ chấm: 4 tấn 562 kg =.........tấn là:\n### Choices: ['A. 45,62','B. 4,562','C. 456,2','D. 4562']\n### Explanation: ", Generated text: 'Để giải quyết câu hỏi này, chúng ta cần hiểu rõ về đơn vị đo lường trong hệ thống đo lường. \n\nTrong hệ thống đo lường, 1 tấn bằng 1000 kg. \n\nVậy nên, khi chúng ta đổi 4 tấn sang đơn vị kg, chúng ta sẽ có 4 tấn * 1000 kg/tấn = 4000 kg. \n\nTiếp theo, chúng ta cần thêm số kg còn lại sau khi đổi sang đơn vị kg. \n\nSố kg còn lại sau khi đổi sang đơn vị kg là 562 kg. \n\nVậy, số thập phân thích hợp để điền vào chỗ chấm là 4000 kg + 562 kg = 4562 kg. \n\nVì vậy, đáp án chính xác là 4562.\n### Final choice: D. 4562\n'
full answer process D. 4562


Processed prompts:   0%|                                                                                                                                                                     | 0/1 [00:00<?, ?it/s]